<a href="https://colab.research.google.com/github/Resia05/unsupervised_learning/blob/main/%D0%9A%D0%BE%D0%BF%D1%96%D1%8F_%D0%B7%D0%B0%D0%BF%D0%B8%D1%81%D0%BD%D0%B8%D0%BA%D0%B0_%22HW_%D0%9C%D0%B5%D1%82%D0%BE%D0%B4%D0%B8_%D0%BF%D0%BE%D0%BD%D0%B8%D0%B6%D0%B5%D0%BD%D0%BD%D1%8F_%D1%80%D0%BE%D0%B7%D0%BC%D1%96%D1%80%D0%BD%D0%BE%D1%81%D1%82%D1%96_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Домашнє завдання: Пониження розмірностей для Аналізу Портретів Клієнтів

#### Контекст
В цьому ДЗ ми попрацюємо з методами пониження розмірності на наборі даних для задачі аналізу портретів клієнтів (Customer Personality Analysis). **В попередньому ДЗ ми працювали з цими даними використовуючи кластеризацію, зараз використаємо кластеризацію і візуалізауємо результати з різними методами.**

Customer Personality Analysis - це аналіз різних сегментів клієнтів компанії. Цей аналіз дозволяє бізнесу краще розуміти своїх клієнтів і полегшує процес адаптації продуктів під конкретні потреби, поведінку та інтереси різних типів клієнтів.

Аналіз портретів клієнтів допомагає бізнесу змінювати свій продукт на основі цільової аудиторії, розділеної на різні сегменти. Наприклад, замість того, щоб витрачати гроші на маркетинг нового продукту для всіх клієнтів у базі даних компанії, бізнес може проаналізувати, який сегмент клієнтів найімовірніше придбає продукт, і потім зосередити маркетингові зусилля лише на цьому сегменті.

#### Вхідні дані
Вам надано набір даних з такими атрибутами:

**Характеристики користувачів:**
- `ID`: Унікальний ідентифікатор клієнта
- `Year_Birth`: Рік народження клієнта
- `Education`: Рівень освіти клієнта
- `Marital_Status`: Сімейний стан клієнта
- `Income`: Річний дохід домогосподарства клієнта
- `Kidhome`: Кількість дітей у домогосподарстві клієнта
- `Teenhome`: Кількість підлітків у домогосподарстві клієнта
- `Dt_Customer`: Дата реєстрації клієнта у компанії
- `Recency`: Кількість днів з моменту останньої покупки клієнта
- `Complain`: 1, якщо клієнт скаржився за останні 2 роки, 0 - якщо ні

**Продукти:**
- `MntWines`: Сума, витрачена на вино за останні 2 роки
- `MntFruits`: Сума, витрачена на фрукти за останні 2 роки
- `MntMeatProducts`: Сума, витрачена на м'ясні продукти за останні 2 роки
- `MntFishProducts`: Сума, витрачена на рибні продукти за останні 2 роки
- `MntSweetProducts`: Сума, витрачена на солодощі за останні 2 роки
- `MntGoldProds`: Сума, витрачена на золото за останні 2 роки

**Акції:**
- `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
- `AcceptedCmp1`: 1, якщо клієнт прийняв пропозицію у першій кампанії, 0 - якщо ні
- `AcceptedCmp2`: 1, якщо клієнт прийняв пропозицію у другій кампанії, 0 - якщо ні
- `AcceptedCmp3`: 1, якщо клієнт прийняв пропозицію у третій кампанії, 0 - якщо ні
- `AcceptedCmp4`: 1, якщо клієнт прийняв пропозицію у четвертій кампанії, 0 - якщо ні
- `AcceptedCmp5`: 1, якщо клієнт прийняв пропозицію у п'ятій кампанії, 0 - якщо ні
- `Response`: 1, якщо клієнт прийняв пропозицію в останній кампанії, 0 - якщо ні

**Взаємодія з компанією:**
- `NumWebPurchases`: Кількість покупок, зроблених через вебсайт компанії
- `NumCatalogPurchases`: Кількість покупок, зроблених за каталогом
- `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
- `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць


Для початку, запустіть код нижче. Всі ці кроки ми робили в попередньому ДЗ і для того, щоб результати кластеризації у нас були схожими, потрібно аби передобробка була однаковою.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# 1. Завантаження даних
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/marketing_campaign.csv', sep='\t')

# 2. Обробка пропущених значень
df['Income_not_filled'] = df.Income.isna()
df.Income = df.Income.fillna(-1)

# 3. Обробка дати реєстрації
df.Dt_Customer = pd.to_datetime(df.Dt_Customer, format='%d-%m-%Y')
today = df.Dt_Customer.max()
df['days_lifetime'] = (today - df.Dt_Customer).dt.days
df['years_customer'] = df.Year_Birth.apply(lambda x: today.year - x)

# 4. Категоризація рівня освіти
df_education = pd.get_dummies(df.Education, prefix='education').astype(int)
df = pd.concat([df, df_education], axis=1)

# 5. Очищення сімейного стану
marital_status_map = {'Alone': 'Single', 'Absurd': 'Else', 'YOLO': 'Else'}
df['Marital_Status_clean'] = df.Marital_Status.map(marital_status_map)
df_ms = pd.get_dummies(df.Marital_Status_clean, prefix='marital').astype(int)
df = pd.concat([df, df_ms], axis=1)

# 6. Форматування доходу і видалення викиду
df.Income = df.Income.astype(int)
df = df[df.Income != 666666]

# 7. Створення фінального набору даних
X = df.drop(['ID', 'Dt_Customer', 'Education', 'Marital_Status', 'Marital_Status_clean'], axis=1)
X.reset_index(drop=True, inplace=True)

### Завдання 1: Виконання кластеризації та пониження розмірності для візуалізації результатів

Ваше завдання — провести кластеризацію клієнтів та візуалізувати результати кластеризації, використовуючи метод головних компонент (PCA) для пониження розмірності даних.

#### Інструкції:

1. **Вибір ключових характеристик:**
   Давайте обмежимось тільки наступними хараткеристиками для кластеризації цього разу:
   - `Income`: Річний дохід домогосподарства клієнта
   - `Recency`: Кількість днів з моменту останньої покупки клієнта
   - `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
   - `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
   - `days_lifetime`: Кількість днів з моменту реєстрації клієнта у компанії
   - `years_customer`: Вік клієнта
   - `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць
   Відберіть в наборі даних `X` лише ці характеристики.

2. **Нормалізація даних:**
   Використайте метод `MinMaxScaler` для нормалізації значень обраних характеристик.

3. **Кластеризація:**
   Проведіть кластеризацію клієнтів, використовуючи метод `KMeans` з трьома кластерами.

4. **Пониження розмірності:**
   Використайте метод головних компонент (PCA) для пониження розмірності даних до трьох компонент.

5. **Візуалізація результатів:**
   Використовуючи plolty express побудуйте 3D-графік розподілу клієнтів у просторі трьох головних компонент, де кольором позначено кластери.

6. **Опишіть, що спостерігаєте:**
   Чи кластеризація чітко розділила дані?

Далі ми детальніше проінтерпретуємо результати візуалізації і пониження розмірностей.

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px

# Вибір ключових характеристик для кластеризації
features = ['Income', 'Recency', 'NumStorePurchases', 'NumDealsPurchases',
            'days_lifetime', 'years_customer', 'NumWebVisitsMonth']
X = df[features]

# Нормалізація даних
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Кластеризація за допомогою KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)
df['Cluster'] = clusters

# Пониження розмірності за допомогою PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
df['PCA1'] = X_pca[:, 0]
df['PCA2'] = X_pca[:, 1]
df['PCA3'] = X_pca[:, 2]

# Візуалізація кластерів у просторі головних компонент
fig = px.scatter_3d(df, x='PCA1', y='PCA2', z='PCA3', color='Cluster',
                    title='Кластеризація клієнтів у просторі PCA',
                    labels={'PCA1': 'Головна компонента 1', 'PCA2': 'Головна компонента 2', 'PCA3': 'Головна компонента 3'})
fig.show()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Здалеку виглядає, наче кластеризація відбулась ідеально. Але якщо приблизитись і розглянути детально, то все-таки є деякі перетини між кластерами. Вони знаходяться майже на межі значень. Загалом поділ відбувся добре - кластери утворили чіткі групи.

### Завдання 2: Аналіз результатів пониження розмірності

1. **Розрахунок частки поясненої дисперсії:**
   Визначте, яка частка загальної варіації даних пояснюється кожною з трьох головних компонент (PC1, PC2, PC3) за допомогою атрибуту `explained_variance_ratio_` об'єкта PCA. Виведіть результат на екран.

2. **Розрахунок кумулятивної частки поясненої дисперсії:**
   Обчисліть кумулятивну частку поясненої дисперсії для трьох головних компонент, щоб зрозуміти, скільки варіації даних пояснюється першими кількома компонентами.

In [5]:
# Розрахунок частки поясненої дисперсії для кожної з трьох головних компонент
explained_variance = pca.explained_variance_ratio_

# Виведення частки поясненої дисперсії для кожної компоненти
print("Частка поясненої дисперсії для кожної компоненти:")
print(f"PC1: {explained_variance[0]:.4f}")
print(f"PC2: {explained_variance[1]:.4f}")
print(f"PC3: {explained_variance[2]:.4f}")

# Кумулятивна частка поясненої дисперсії
cumulative_explained_variance = explained_variance.cumsum()
print("\nКумулятивна частка поясненої дисперсії:")
print(f"PC1 + PC2: {cumulative_explained_variance[1]:.4f}")
print(f"PC1 + PC2 + PC3: {cumulative_explained_variance[2]:.4f}")


Частка поясненої дисперсії для кожної компоненти:
PC1: 0.3020
PC2: 0.2867
PC3: 0.2512

Кумулятивна частка поясненої дисперсії:
PC1 + PC2: 0.5887
PC1 + PC2 + PC3: 0.8399


### Завдання 3: Інтерпретація "Loadings"

Продовжуємо інтерпретацію результатів `PCA`і познайомимось з новим поняттям `loadings`, яке допоможе нам знайти звʼязок між головними компонентами і оригінальними ознаками в наборі даних.

Ми зараз побудували візуалізацію кластерів точок даних в просторі трьох головних компонент. Але хочеться знайти звʼязок між головними компонентами і оригінальними ознаками. Для розуміння, які початкові характеристики даних мають найбільший вплив на ці головні компоненти, ми можемо використати атрибут `components_` методу `PCA`.

#### Що таке `pca.components_`?

`pca.components_` — це масив, який містить коефіцієнти (або "ваги"), що показують внесок кожної вихідної ознаки у кожну з головних компонент. Ці коефіцієнти ще називаються **"loading"** або "навантаженнями" компонент.

- **Loadings** (`навантаження`) відображають важливість кожної змінної (ознаки) для відповідної головної компоненти. Вони показують, яким чином змінні поєднуються, щоб утворити нові, зменшені вимірювання.
- Якщо коефіцієнт має високе абсолютне значення (як позитивне, так і негативне), це вказує на те, що відповідна змінна сильно впливає на головну компоненту.

#### Саме завдання
Ваше завдання — обчислити "навантаження" для кожної з головних компонент і інтерпретувати результати.

1. **Обчислення loadings для компонент:**
   Використайте атрибут `components_` об'єкта PCA для створення DataFrame, який відображатиме внесок кожної вихідної ознаки в кожну головну компоненту.

2. **Інтерпретація результатів:**
   Виведіть значення "навантажень" і проаналізуйте, які ознаки найбільше впливають на кожну головну компоненту.

In [6]:
# Обчислюємо loadings для кожної компоненти
loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', 'PC3'], index=features)

loadings

,PC1,PC2,PC3
Income,0.063557,0.047082,-0.373826
Recency,0.475786,-0.878876,-0.029910
NumStorePurchases,0.284282,0.187195,-0.816668
NumDealsPurchases,0.103049,0.059324,0.050687
days_lifetime,0.821912,0.431061,0.305490
years_customer,0.012658,-0.001022,-0.080411
NumWebVisitsMonth,0.049530,0.031377,0.300089


Для PC1 найбільший вплив мають days_lifetime, Recency - к-сть днів з моменту реєстрації та покупки.

Для PC2 найбільший вплив мають Recency, days_lifetime	- теж к-сть днів, але з від'ємним значенням, тобто неактивні клієнти.

Для PC3 найбільший вплив мають NumStorePurchases, days_lifetime - тут ще додається вплив від к-сті покупок.

###Завдання 4
Давайте проаналізуємо "навантаження" (**loadings**) для трьох головних компонент після вилучення ознаки `Income`. Це допоможе нам зрозуміти, як змінилася важливість інших ознак для кожної головної компоненти, коли одна з ключових характеристик (`Income`) була вилучена.

#### Кроки для проведення аналізу і ваше завдання:

1. Видаліть ознаку `Income` з нашого набору даних `X` і повторно виконайте PCA (метод головних компонент) для отримання нових "навантажень".

2. Обчисліть нові "навантаження" для трьох головних компонент на наборі даних без `Income`

3. Проаналізуйте, які ознаки мають найбільший вплив на кожну головну компоненту після вилучення `Income`.

4. Перегляньте, наскільки кожна з головних компонент пояснює дисперсію в даних без ознаки `Income`.

In [7]:
# 1. Видалення ознаки Income
X_no_income = X.drop(columns=['Income'])
X_no_income

,Recency,NumStorePurchases,NumDealsPurchases,days_lifetime,years_customer,NumWebVisitsMonth
0,58,4,3,663,57,7
1,38,2,2,113,60,5
2,26,10,1,312,49,4
3,26,4,2,139,30,6
4,94,6,5,161,33,5
...,...,...,...,...,...,...
2235,46,4,2,381,47,5
2236,56,5,7,19,68,7
2237,91,13,1,155,33,6
2238,8,10,2,156,58,3


In [8]:
# 2. Нормалізація даних
scaler_no_income = MinMaxScaler()
X_scaled_no_income = scaler_no_income.fit_transform(X_no_income)

# 3. Виконання PCA
pca_no_income = PCA(n_components=3)
pca_no_income.fit(X_scaled_no_income)

# 4. Отримання "навантажень"
loadings_no_income = pd.DataFrame(pca_no_income.components_.T,
                                  columns=['PC1', 'PC2', 'PC3'],
                                  index=X_no_income.columns)

# 5. Виведення навантажень
print("Loadings після вилучення ознаки Income:")
print(loadings_no_income)

# 6. Виведення частки поясненої дисперсії
explained_variance_ratio_no_income = pca_no_income.explained_variance_ratio_
print("\nЧастка поясненої дисперсії для трьох головних компонент (без Income):")
print(explained_variance_ratio_no_income)

# 7. Кумулятивна частка поясненої дисперсії
cumulative_variance_no_income = explained_variance_ratio_no_income.cumsum()
print("\nКумулятивна частка поясненої дисперсії (без Income):")
print(cumulative_variance_no_income)

Loadings після вилучення ознаки Income:
                        PC1       PC2       PC3
Recency            0.500712 -0.865392 -0.008648
NumStorePurchases  0.226953  0.145537 -0.924549
NumDealsPurchases  0.105012  0.065833  0.027582
days_lifetime      0.825793  0.472036  0.232275
years_customer     0.007110 -0.005913 -0.079808
NumWebVisitsMonth  0.069051  0.052233  0.289919

Частка поясненої дисперсії для трьох головних компонент (без Income):
[0.32149032 0.30545488 0.23446936]

Кумулятивна частка поясненої дисперсії (без Income):
[0.32149032 0.6269452  0.86141456]


Нові "навантаження" (loadings):
PC1: days_lifetime має максимальний вплив. Окрім цього, значний вплив має також Recency, як і в попередньому аналізі.
PC2: найбільший вплив має ознака Recency, що свідчить про її важливість у протилежному напрямку до інших ознак. Також відчутний вплив спостерігається у ознаки days_lifetime. Результати теж схожі. як і впопередньому аналізі з Income.
PC3: Найбільший вплив на третю компоненту після вилучення Income має ознака NumStorePurchases.

Перші три компоненти без ознаки Income пояснюють 86.14% варіації у даних, що є доволі високим показником.

Незважаючи на вилучення Income, модель все ще може добре пояснювати більшу частину дисперсії в даних.


### Завдання 5: Візуалізація кластеризації за допомогою t-SNE

Ваше завдання — використати метод t-SNE для візуалізації результатів кластеризації клієнтів у двовимірному просторі. Метод t-SNE допомагає знизити розмірність даних та зберегти локальні структури в даних, що робить його ефективним для візуалізації високорозмірних даних. Ми також зможемо порівняти результат цього методу з РСА.

1. Використайте метод t-SNE для зниження розмірності до 2х вимірів даних, які включають ознаки всі, що і в завданні 1, а також були відмасштабовані перед пониженням розмірностей.

2. Створіть новий DataFrame з координатами, отриманими після застосування t-SNE, та додайте до нього мітки кластерів.

3. Побудуйте інтерактивний 2D-графік розподілу клієнтів, де кольором буде позначено різні кластери і проаналізуйте графік з рекомендаціями нижче (можливо треба буде вивести додаткові візуалізації чи таблиці для інтерпретації, але треба прям зрозуміти, які ознаки формують який кластер і чим кластери відрізняються одне від одного).

  **Опишіть отримані кластери з точки зору ознак.**

4. Опишіть відмінність графіка tSNE від PCA.

#### ЯК можна інтерпретувати з t-SNE?

Хоча t-SNE не надає "компонентів" як РСА, він забезпечує низьковимірне представлення даних, яке можна візуально інтерпретувати:

- **Кластери:** t-SNE особливо добре показує кластери подібних точок. Якщо ви бачите чітко визначені кластери на графіку t-SNE, це свідчить про наявність груп схожих спостережень у ваших даних. Проаналізувати їх можемо, якщо додамо дані в `hover_data` або якщо якісь з даних виведемо як розмір чи форма точок на візуалізації. Також корисно може бути вивести середні значення ознак по кластерам.
- **Локальна структура:** Відносне розташування точок одного кластеру на графіку t-SNE може допомогти вам зрозуміти, які дані подібні між собою.
- **Глобальна структура:** Будьте обережні; t-SNE менш надійний для відображення глобальних структур (наприклад, відстаней між кластерами) у порівнянні з PCA, бо t-SNE націлений на збереження саме локальних структур.

In [12]:
from sklearn.manifold import TSNE

# Застосування t-SNE для зниження до 2 вимірів
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_scaled)

# Створення DataFrame з координатами t-SNE і додавання міток кластерів
df_tsne = pd.DataFrame(X_tsne, columns=['t-SNE1', 't-SNE2'])
df_tsne['Cluster'] = clusters  # Додаємо кластери, які були отримані раніше

# Додаємо самі значення ознак в DataFrame для hover_data
df_tsne = pd.concat([df_tsne, X.reset_index(drop=True)], axis=1)

# Створення інтерактивного графіка t-SNE
fig = px.scatter(df_tsne, x='t-SNE1', y='t-SNE2', color='Cluster', hover_data=X.columns)
fig.update_layout(title="t-SNE Visualization of Clusters")
fig.show()

# Виведення середніх значень ознак по кластерах
cluster_means = X.groupby(clusters).mean()
print(cluster_means)


         Income    Recency  NumStorePurchases  NumDealsPurchases  \
0  52835.119048  36.483193           6.726891           3.012605   
1  50349.671951  79.179268           5.362195           2.100000   
2  51208.812766  26.958865           5.343262           1.887943   

   days_lifetime  years_customer  NumWebVisitsMonth  
0     553.358543       45.088235           5.876751  
1     328.232927       45.441463           5.239024  
2     180.685106       45.025532           4.838298  


In [20]:
fig = px.scatter_3d(df, x='PCA1', y='PCA2', z='PCA3', color='Cluster',
                    title='Кластеризація клієнтів у просторі PCA',
                    labels={'PCA1': 'Головна компонента 1', 'PCA2': 'Головна компонента 2', 'PCA3': 'Головна компонента 3'})
#fig.update_layout(    height=900,     width=1600, ) -  налаштування під мій екран ноута
fig.show()

Кластер 0: Цей кластер характеризується високими значеннями ознаки NumStorePurchases (в середньому 6.73) та найвищим середнім значенням days_lifetime (553.36), що вказує на те, що клієнти в цьому кластері є активними покупцями і залишаються клієнтами тривалий час. Водночас, клієнти мають високе середнє значення за кількістю відвідувань вебсайту (5.88).

Кластер 1: У цьому кластері помітно високий рівень ознаки Recency (79.18), що вказує на те, що ці клієнти нещодавно здійснювали покупки. У них також найменше значення ознаки NumDealsPurchases (2.10), що може вказувати на те, що вони менше користуються знижками та акціями.

Кластер 2: Цей кластер має найменше середнє значення за days_lifetime (180.69), що може вказувати на те, що клієнти в цьому кластері є новішими, порівняно з іншими кластерами. Крім того, вони також мають найменше середнє значення за кількістю відвідувань вебсайту (4.84), що може вказувати на нижчу активність.


При аналізі результатів кластеризації з використанням t-SNE, візуалізація показує, що дані мають більш розкидану структуру у порівнянні з результатами PCA. Кластери у просторі t-SNE менш чітко окреслені, що може свідчити про меншу виразність границь між групами у цьому низьковимірному представленні.

Також, t-SNE дозволяє більш явно виявити аутлаєри, які виглядають так, ніби вони належать до зовсім іншого кластера. Ці точки розміщені далеко від основних груп і часто мають тенденцію бути відокремленими, тоді як при використанні PCA такі аномальні спостереження не були настільки очевидні.

Ці відмінності візуалізації підкреслюють, що t-SNE зберігає локальні структури даних і краще справляється з виявленням аутлаєрів, хоча глобальна структура може бути менш чітко представлена в порівнянні з PCA.